## **This Notebook will construct the nodes and edges that we will build our graph with**

Import relevant libraries and import raw data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/current_clean_spectrum.csv")

In [3]:
#sns.lineplot(data=df, x='WaveNumber', y='Absorbance', hue='Status', errorbar=('ci', False))

In [4]:
df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
0,201210-1-00,293,400.22778,201210-1,Normal,41.863303
1,201210-1-00,294,400.91116,201210-1,Normal,41.803843
2,201210-1-00,295,401.59454,201210-1,Normal,41.741884
3,201210-1-00,296,402.27789,201210-1,Normal,41.677722
4,201210-1-00,297,402.96127,201210-1,Normal,41.611654
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,12.378163
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,13.269937
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,14.199285
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,15.166531


Set each WaveNumber as a column

In [5]:
pivoted_df = df.pivot(index='SpecID', columns='WaveNumber', values='Absorbance').reset_index()
pivoted_df.columns.name = None

statuses = df[['SpecID', 'Status']].drop_duplicates()

pivoted_df = pd.merge(pivoted_df, statuses, on='SpecID')
pivoted_df = pivoted_df.set_index('SpecID')

pivoted_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.3053,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,41.863303,41.803843,41.741884,41.677722,41.611654,41.543974,41.474980,41.404968,41.334234,41.263073,...,6.942749,6.280946,5.549559,4.745724,3.866578,2.909255,1.870891,0.748623,-0.460415,Normal
201210-1-01,46.314608,47.323684,48.299209,49.241395,50.150457,51.026608,51.870063,52.681035,53.459738,54.206386,...,6.322524,6.769011,7.280928,7.861246,8.512936,9.238972,10.042323,10.925962,11.892860,Normal
201210-1-02,118.159018,114.686240,111.563911,108.777452,106.312282,104.153823,102.287493,100.698715,99.372907,98.295491,...,-4.668874,-4.633601,-4.557349,-4.439365,-4.278894,-4.075180,-3.827470,-3.535010,-3.197043,Normal
201210-1-03,175.466997,174.846086,174.188020,173.498226,172.782129,172.045155,171.292728,170.530275,169.763222,168.996993,...,-11.220152,-10.801936,-10.349539,-9.864191,-9.347124,-8.799567,-8.222752,-7.617909,-6.986269,Normal
201210-1-04,111.814973,106.629998,101.867380,97.512673,93.551430,89.969205,86.751551,83.884023,81.352173,79.141556,...,-11.600625,-11.689508,-11.752441,-11.789205,-11.799583,-11.783357,-11.740310,-11.670224,-11.572882,Normal


In [6]:
status_df = df.groupby('SpecID')[['SurID', 'Status']].first()
# surface_and_status.to_csv("../../data/surface_and_status.csv")
status_df

,SurID,Status
SpecID,,
201210-1-00,201210-1,Normal
201210-1-01,201210-1,Normal
201210-1-02,201210-1,Normal
201210-1-03,201210-1,Normal
201210-1-04,201210-1,Normal
...,...,...
210526-3-45,210526-3,Hyperglycemia
210526-3-46,210526-3,Hyperglycemia
210526-3-47,210526-3,Hyperglycemia


>#### **Calculate an adjacency matrix using a Gaussian Kernel**

In [7]:
from scipy.spatial.distance import pdist, squareform

# Drop the 'Status' column as it is not numeric
X = pivoted_df.drop(columns=['Status'])

# Calculate pairwise Euclidean distances
distances = pdist(X.values, metric='euclidean')

# Convert the condensed distances to a square matrix form
distance_matrix = squareform(distances)

In [8]:
def gaussian_kernel(distances, sigma):
    return np.exp(-distances**2 / (2 * sigma**2))

# Sigma is the bandwidth parameter
sigma = 3000 # Adjust sigma as needed
kernel_matrix = gaussian_kernel(distance_matrix, sigma)

# Convert the kernel matrix to a DataFrame
kernel_df = pd.DataFrame(kernel_matrix, index=pivoted_df.index, columns=pivoted_df.index)

In [9]:
display(kernel_df.head())
kernel_df.shape

SpecID,201210-1-00,201210-1-01,201210-1-02,201210-1-03,201210-1-04,201210-1-05,201210-1-06,201210-1-07,201210-1-09,201210-1-10,...,210526-3-40,210526-3-41,210526-3-42,210526-3-43,210526-3-44,210526-3-45,210526-3-46,210526-3-47,210526-3-48,210526-3-49
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1.000000e+00,8.501310e-01,0.001195,7.543848e-08,4.992110e-01,2.902027e-01,2.881005e-01,4.868298e-01,5.756645e-01,7.103626e-01,...,7.768656e-01,7.732815e-01,7.422501e-01,7.693990e-01,7.783326e-01,7.674556e-01,7.626145e-01,7.529947e-01,7.638640e-01,7.565100e-01
201210-1-01,8.501310e-01,1.000000e+00,0.002901,2.442991e-07,6.248388e-01,4.033422e-01,3.709988e-01,5.298339e-01,5.943973e-01,6.586015e-01,...,6.185016e-01,6.234690e-01,5.869408e-01,5.935506e-01,5.913871e-01,5.758084e-01,5.712532e-01,5.542056e-01,5.664134e-01,5.594924e-01
201210-1-02,1.195401e-03,2.900821e-03,1.000000,7.763462e-05,1.203051e-03,1.036456e-03,6.822934e-04,9.328108e-04,1.155250e-03,8.528864e-04,...,3.237659e-04,3.413237e-04,3.091399e-04,2.986238e-04,2.924107e-04,2.707459e-04,2.774479e-04,3.050866e-04,2.752349e-04,2.623500e-04
201210-1-03,7.543848e-08,2.442991e-07,0.000078,1.000000e+00,1.406647e-07,1.726678e-07,1.864192e-07,1.759841e-07,1.968682e-07,8.338672e-08,...,2.070781e-08,2.445093e-08,1.793387e-08,1.550976e-08,1.416413e-08,1.203312e-08,1.171406e-08,1.260120e-08,1.224969e-08,1.140751e-08
201210-1-04,4.992110e-01,6.248388e-01,0.001203,1.406647e-07,1.000000e+00,7.967328e-01,4.951024e-01,5.256507e-01,5.169731e-01,5.403555e-01,...,4.000734e-01,4.039161e-01,3.719100e-01,3.704179e-01,3.547518e-01,3.428757e-01,3.424278e-01,3.191699e-01,3.281033e-01,3.343850e-01


(3045, 3045)

**Construct the node and edge import files with the required Admin Import Headers**

In [10]:
relationship_df = kernel_df.reset_index()
relationship_df = relationship_df.melt(id_vars='SpecID', var_name=':END_ID', value_name='Weight:float')
relationship_df = relationship_df.rename(columns={'SpecID': ':START_ID'})
relationship_df[':TYPE'] = 'LINK'

Drop Relationships a node has with itself, along with repeat relationships.

In [11]:
relationship_df = relationship_df[relationship_df[":START_ID"] != relationship_df[":END_ID"]]
relationship_df = relationship_df[relationship_df[":START_ID"] < relationship_df[":END_ID"]]  # Drops redundant relationships
relationship_df

,:START_ID,:END_ID,Weight:float,:TYPE
3045,201210-1-00,201210-1-01,8.501310e-01,LINK
6090,201210-1-00,201210-1-02,1.195401e-03,LINK
6091,201210-1-01,201210-1-02,2.900821e-03,LINK
9135,201210-1-00,201210-1-03,7.543848e-08,LINK
9136,201210-1-01,201210-1-03,2.442991e-07,LINK
...,...,...,...,...
9272019,210526-3-44,210526-3-49,9.849016e-01,LINK
9272020,210526-3-45,210526-3-49,9.846659e-01,LINK
9272021,210526-3-46,210526-3-49,9.888397e-01,LINK
9272022,210526-3-47,210526-3-49,9.829190e-01,LINK


In [12]:
node_df = status_df.drop(columns=['SurID']).reset_index()
node_df.rename(columns={'SpecID':'SpecID:ID', 'Status':'Status:LABEL'}, inplace=True)
node_df

,SpecID:ID,Status:LABEL
0,201210-1-00,Normal
1,201210-1-01,Normal
2,201210-1-02,Normal
3,201210-1-03,Normal
4,201210-1-04,Normal
...,...,...
3040,210526-3-45,Hyperglycemia
3041,210526-3-46,Hyperglycemia
3042,210526-3-47,Hyperglycemia
3043,210526-3-48,Hyperglycemia


**Set an Environmental Variable as your working Neo4J directory for this to work.**

In [13]:
import os

Neo4j_Directory = os.environ['NEO4J_Directory']
print(Neo4j_Directory)

/home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a


Save the nodes and relationships files in the import folder.

In [14]:
node_df.to_csv(Neo4j_Directory + '/import/nodes.csv')
relationship_df.to_csv(Neo4j_Directory + '/import/relationships.csv')

In [15]:
relationship_df

,:START_ID,:END_ID,Weight:float,:TYPE
3045,201210-1-00,201210-1-01,8.501310e-01,LINK
6090,201210-1-00,201210-1-02,1.195401e-03,LINK
6091,201210-1-01,201210-1-02,2.900821e-03,LINK
9135,201210-1-00,201210-1-03,7.543848e-08,LINK
9136,201210-1-01,201210-1-03,2.442991e-07,LINK
...,...,...,...,...
9272019,210526-3-44,210526-3-49,9.849016e-01,LINK
9272020,210526-3-45,210526-3-49,9.846659e-01,LINK
9272021,210526-3-46,210526-3-49,9.888397e-01,LINK
9272022,210526-3-47,210526-3-49,9.829190e-01,LINK


-----

#### **This Section Creates the Graph in Neo4J then runs the Queries.**

In [16]:
from neo4j import GraphDatabase
import subprocess

**Run the admin import command to create the graph from the CSVs**

In [17]:
# Construct the command Linux
command = [
    './bin/neo4j-admin', 'database', 'import', 'full',
    '--nodes=import/nodes.csv',
    '--relationships=import/relationships.csv',
    '--overwrite-destination',  'neo4j'
]

#Windows
#command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv --overwrite-destination neo4j'
#result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)

result = subprocess.run(command, cwd=Neo4j_Directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
b"Neo4j version: 5.12.0\nImporting the contents of these files into /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data/databases/neo4j:\nNodes:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/nodes.csv\n\nRelationships:\n  /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\n\nAvailable resources:\n  Total machine memory: 13.49GiB\n  Free machine memory: 210.1MiB\n  Max heap memory : 910.5MiB\n  Max worker threads: 12\n  Configured max memory: 71.23MiB\n  High parallel IO: true\n\nCypher type normalization is enabled (disable with --normalize-types=false):\n  Property type of 'Weight' normalized from 'float' --> 'double' in /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import/relationships.csv\n\nImport 

Restart the database for the changes to take effect


In [18]:
command = [
    './bin/neo4j', 'restart'
]

result = subprocess.run(command, cwd=Neo4j_Directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Restart successful")
    print(result.stdout)
else:
    print("Error during restart:")
    print(result.stderr)

Restart successful
b'Stopping Neo4j........ stopped.\nConfiguration file validation successful (with warnings).\nDirectories in use:\nhome:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a\nconfig:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/conf\nlogs:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/logs\nplugins:      /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/plugins\nimport:       /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/import\ndata:         /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fde3105cb8a/data\ncertificates: /home/eddie/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-e4c1e17b-4cb7-4b84-a9fb-3fd

Run and connect to the Neo4j Database

In [19]:
uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
password = os.environ['NEO4J_Password']           # Neo4J password'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [20]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Unable to retrieve routing information
Transaction failed and will be retried in 0.8546569494434144s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 1.603450614017559s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 3.4648957659226722s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 9.068924896992723s (Unable to retrieve routing information)


##### **Define the graph algorithms.**

In [21]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [22]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [23]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [24]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [25]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [26]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [27]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [28]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [29]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'Weight',
         randomSeed:1234,
         embeddingDimension: 512
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

##### **Execute the Graph Algorithms and Store the Results**

In [30]:
# # Use a session to execute the queries and retrieve the results
# with driver.session() as session:
#     pagerank_results = session.execute_read(run_pagerank_centrality)
#     degree_results = session.execute_read(run_degree_centrality)
#     eigenvector_results = session.execute_read(run_eigenvector_centrality)
#     articlerank_results = session.execute_read(run_articlerank_centrality)
#     label_propagation_results = session.execute_read(run_label_propagation_algorithm)
#     leiden_results = session.execute_read(run_leiden_algorithm)
#     louvain_results = session.execute_read(run_louvain_algorithm)

# pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
# degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
# eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
# articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
# label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
# leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
# louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

# merged_df = pagerank_df
# for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
#     merged_df = pd.merge(merged_df, df, on=['name'], how='left')

# graph_metrics_df = merged_df.rename(columns={'name' : 'SpecID'})
# graph_metrics_df

##### **Get Node Embeddings using FastRP**

In [31]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_502,embedding_503,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511
0,201210-1-00,-0.088389,0.020422,0.147291,0.141107,0.076047,-0.051798,0.008347,-0.016279,-0.020508,...,0.109565,-0.044653,0.020160,0.085649,0.096982,-0.011981,0.028314,-0.156349,-0.055221,0.084925
1,201210-1-01,-0.083970,0.021639,0.146236,0.138482,0.078607,-0.051480,0.008602,-0.018410,-0.018800,...,0.109995,-0.042703,0.023753,0.083272,0.095447,-0.012564,0.025295,-0.154007,-0.052783,0.087279
2,201210-1-02,-0.072841,0.024081,0.144375,0.132327,0.084038,-0.048699,0.008751,-0.022922,-0.014418,...,0.109759,-0.036794,0.031896,0.076468,0.092639,-0.014681,0.019510,-0.146837,-0.047957,0.093192
3,201210-1-03,-0.018323,0.008027,0.127082,0.082085,0.094985,-0.022637,-0.000978,-0.060352,0.024082,...,0.097891,0.027238,0.083069,-0.001681,0.096954,-0.018749,0.024557,-0.075894,-0.007323,0.143794
4,201210-1-04,-0.077046,0.023361,0.144474,0.133169,0.081799,-0.051000,0.009235,-0.021873,-0.015591,...,0.111778,-0.038459,0.030767,0.077876,0.092512,-0.012997,0.022328,-0.149724,-0.048062,0.092219


In [32]:
fastRP_Status = fastRP_df.merge(status_df, on='SpecID')
fastRP_Status

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511,SurID,Status
0,201210-1-00,-0.088389,0.020422,0.147291,0.141107,0.076047,-0.051798,0.008347,-0.016279,-0.020508,...,0.020160,0.085649,0.096982,-0.011981,0.028314,-0.156349,-0.055221,0.084925,201210-1,Normal
1,201210-1-01,-0.083970,0.021639,0.146236,0.138482,0.078607,-0.051480,0.008602,-0.018410,-0.018800,...,0.023753,0.083272,0.095447,-0.012564,0.025295,-0.154007,-0.052783,0.087279,201210-1,Normal
2,201210-1-02,-0.072841,0.024081,0.144375,0.132327,0.084038,-0.048699,0.008751,-0.022922,-0.014418,...,0.031896,0.076468,0.092639,-0.014681,0.019510,-0.146837,-0.047957,0.093192,201210-1,Normal
3,201210-1-03,-0.018323,0.008027,0.127082,0.082085,0.094985,-0.022637,-0.000978,-0.060352,0.024082,...,0.083069,-0.001681,0.096954,-0.018749,0.024557,-0.075894,-0.007323,0.143794,201210-1,Normal
4,201210-1-04,-0.077046,0.023361,0.144474,0.133169,0.081799,-0.051000,0.009235,-0.021873,-0.015591,...,0.030767,0.077876,0.092512,-0.012997,0.022328,-0.149724,-0.048062,0.092219,201210-1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210526-3-45,-0.091401,0.019449,0.147800,0.142272,0.074101,-0.052247,0.008276,-0.015094,-0.021373,...,0.018374,0.086775,0.097710,-0.011213,0.030401,-0.157729,-0.056315,0.083805,210526-3,Hyperglycemia
3041,210526-3-46,-0.091623,0.019437,0.147861,0.142346,0.073858,-0.052321,0.008249,-0.014975,-0.021440,...,0.018272,0.086762,0.097698,-0.011187,0.030760,-0.157841,-0.056374,0.083749,210526-3,Hyperglycemia
3042,210526-3-47,-0.092270,0.019280,0.148023,0.142648,0.073370,-0.052377,0.008227,-0.014642,-0.021649,...,0.017848,0.086950,0.097848,-0.011082,0.031457,-0.158153,-0.056663,0.083493,210526-3,Hyperglycemia
3043,210526-3-48,-0.091947,0.019361,0.147960,0.142524,0.073604,-0.052334,0.008220,-0.014786,-0.021555,...,0.018043,0.086861,0.097787,-0.011158,0.031133,-0.157999,-0.056547,0.083600,210526-3,Hyperglycemia


In [33]:
# import seaborn as sns
# sns.scatterplot(data=fastRP_Status, x='embedding_0', y='embedding_1', hue='Status')

**Delete the projection and close the session**

In [34]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)
    
close_driver()